# **Univariate Analysis**

**Datasets**

*   Immigration - Yearly - 2012 - 2021
*   Mortgage - Daily - 1971/4/2 - latest business day (plot with recession periods)
*   Home Value Index - Monthly - 2000/1 - 2023/4
*   Median Sale Price - Monthly - 2018/8 - 2023/4


### Dependencies and Imports

In [ ]:
# install data profiling tool
!pip install -U ydata-profiling

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.9/352.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.5/679.5 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.4/455.4 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 77.0 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=e5f5da649d1f937fb4117de927f6fba4a8055334ee87653475e2a5086bdb52f7
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin
  Attempting uninstall: wordcloud
    Found existing installation: wordcloud 1.8.2.2
    

In [ ]:
#This is to solve error message "ValueError: Only supported for TrueType fonts"
!pip install --upgrade pip
!pip install --upgrade Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 27.4 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pillow-8.4.0


In [ ]:
# Install 5.0.1 altair version
!pip uninstall -y altair
!pip install altair
#Restart runtime after running the above cells

Found existing installation: altair 4.2.2
Uninstalling altair-4.2.2:
  Successfully uninstalled altair-4.2.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 7.4 MB/s eta 0:00:00


In [ ]:
# Install openpyxl package for reading and writing Excel files (XLSX).
!pip install openpyxl

In [ ]:
# Used for creating visualizations and plots
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import altair as alt
# Used for data manipulation and analysis
import numpy as np
import pandas as pd
# dates and times in Python
import datetime
# Provides functions for interacting with the OS, files, etc.
import requests
import io
import zipfile
import os
# statistical functions for  time series analysis
from statsmodels.tsa.seasonal import seasonal_decompose
# data profiling
from ydata_profiling import ProfileReport
# implements a mathematical technique called folding or reduction.
from functools import reduce

ImportError: ignored

In [ ]:
# Disable the maximum number of rows limit when working with large datasets.
alt.data_transformers.disable_max_rows()
# Enable the default renderer for visualizing Altair plots.
alt.renderers.enable('default')

In [ ]:
assert alt.__version__ == '5.0.1', "Altair version 5.0.1 is required."

### Immigration Data

In [ ]:
immigration_zip_path = 'https://www.dhs.gov/sites/default/files/2022-09/2022_0405_plcy_lawful_permanent_residents_fy2021_excel.zip'
immigration_file_name = 'fy2021_table4.xlsx'

In [ ]:
response = requests.get(immigration_zip_path)
with open('2022_0405_plcy_lawful_permanent_residents_fy2021_excel.zip', 'wb') as zip:
        zip.write(response.content)

with zipfile.ZipFile('2022_0405_plcy_lawful_permanent_residents_fy2021_excel.zip', 'r') as zip_ref:
        zip_ref.extract(immigration_file_name)
immigration_df = pd.read_excel(immigration_file_name, skiprows=3, skipfooter=5)
immigration_df = immigration_df.rename(columns={'State or territory of residence': 'states'})
immigration_df.tail()

In [ ]:
immigration_profile = ProfileReport(immigration_df, title="Immigration Profiling Report")
immigration_profile

In [ ]:
immigration_plot = immigration_df[immigration_df['states']!='Total']
immigration_melt = immigration_plot.melt('states', var_name='year', value_name='count')
states = list(immigration_plot['states'])
states.sort()
dropdown_selection = alt.selection_point(
    name='Select',
    fields=['states'],
    value='Alabama',
    bind=alt.binding_select(options=states)
)
interval_selection = alt.selection_interval(bind='scales', encodings=['y'])

opacityCondition = alt.condition(dropdown_selection, alt.value(0.75), alt.value(0.0))
colorCondition = alt.condition(dropdown_selection, alt.Color('states:N', legend=None), alt.value('white'))

alt.Chart(immigration_melt).mark_line().add_params(
    dropdown_selection,
    interval_selection
).encode(
    x = 'year:N',
    y = alt.Y('count:Q', scale=alt.Scale(domain=[241, 230000])),
    color = colorCondition,
    opacity = opacityCondition
).properties(
    width=800,
    height=300,
    title="Immigration Data"
)

In [ ]:
alt.Chart(immigration_melt).mark_line().encode(
    x = alt.X('year:N', title=None),
    y = alt.Y('count:Q', scale=alt.Scale(type='log')),
    color = alt.Color('states', legend=None),
    tooltip=['states']
).properties(
    width=800,
    height=300,
    title="Immigration Data"
).interactive()

### Mortgage data


In [ ]:
mortgage_rate_path = 'https://www.freddiemac.com/pmms/docs/historicalweeklydata.xlsx'

In [ ]:
response = requests.get(mortgage_rate_path)

mortgage_file = pd.ExcelFile(io.BytesIO(response.content))
mortgage_df = mortgage_file.parse(mortgage_file.sheet_names[0], skiprows=6, skipfooter=1)
mortgage_df = mortgage_df.rename(columns={'Week': 'week',
                                          'FRM': '30yr',
                                          'points': '30yr_f_p',
                                          'FRM.1': '15yr',
                                          'points.1': '15yr_f_p',
                                          'ARM': '5_1_arm',
                                          'points.2': '5_1_arm_f_p',
                                          'margin': '5_1_arm_margin',
                                          'spread': '30yr_5_1_arm_sprd'})
mortgage_df.tail()

In [ ]:
mortgage_profile = ProfileReport(mortgage_df, title="Mortgage Profiling Report")

In [ ]:
mortgage_profile

In [ ]:
mortgage_plot = mortgage_df.copy()
# Convert the 'week' column to datetime
mortgage_plot['week'] = pd.to_datetime(mortgage_plot['week'], errors='coerce')

# Drop rows with invalid datetime values
mortgage_plot = mortgage_plot.dropna(subset=['week'])

# Set the 'week' column as the index
mortgage_plot.set_index('week', inplace=True)

# Perform seasonal decomposition
result_30 = seasonal_decompose(mortgage_plot['30yr'], model='additive', period=52)  # weekly data
result_15 = seasonal_decompose(mortgage_plot['15yr'].dropna(), model='additive', period=52)  # weekly data
result_5_1_arm = seasonal_decompose(mortgage_plot['5_1_arm'].dropna(), model='additive', period=52)  # weekly data
# Plot the decomposition components
result_30.plot()
result_15.plot()
result_5_1_arm.plot()
plt.show()

####Plot Mortgage with Recession Data

In [ ]:
recession_url = 'https://fred.stlouisfed.org/graph/fredgraph.xls?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=off&txtcolor=%23444444&ts=12&tts=12&width=1318&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=JHDUSRGDPBR&scale=left&cosd=1967-10-01&coed=2022-10-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Quarterly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2023-06-09&revision_date=2023-06-09&nd=1967-10-01'

In [ ]:
response = requests.get(recession_url)
recession_file = pd.ExcelFile(io.BytesIO(response.content))
recession_df = recession_file.parse(recession_file.sheet_names[0], skiprows=10)
recession_df = recession_df.rename(columns={'observation_date': 'year_month',
                                            'JHDUSRGDPBR': 'recession'})
recession_df.head()

In [ ]:
recession_df[recession_df['recession']==1]['year_month'].unique()

In [ ]:
mortgage_recession = mortgage_df.copy()
mortgage_recession['month'] = mortgage_recession['week'].dt.to_period('M').dt.to_timestamp()

In [ ]:
mortgage_recession_merge = pd.merge(mortgage_recession, recession_df, left_on='month', right_on='year_month', how='left')
mortgage_recession_merge.head()

In [ ]:
# Create a chart with grey bars
recession_chart = alt.Chart(mortgage_recession_merge).mark_bar(color='lightgrey').encode(
    x=alt.X('week:T', title=''),
    y=alt.Y('recession:Q', axis=alt.Axis(title=''))
)
#recession_chart

In [ ]:
mortgage_recession_merge.loc[mortgage_recession_merge['recession'] == 1, 'recession'] = 20
#30-year fixed rate
mortgage_chart = alt.Chart(mortgage_recession_merge).mark_line().encode(
    x=alt.X('week:T', title=None),
    y=alt.Y('30yr:Q', title='Mortgage Rate %')
)
recession_chart+mortgage_chart.properties(
    width=500,
    height=500,
    title=alt.TitleParams(text = '30 yrs Firm since 1971', fontSize = 22)
)

In [ ]:
mortgage_recession_merge.loc[mortgage_recession_merge['recession'] == 1, 'recession'] = 20
#30-year fixed rate
mortgage_chart = alt.Chart(mortgage_recession_merge).mark_line().encode(
    x=alt.X('week:T', title=None),
    y=alt.Y('15yr:Q', title='Mortgage Rate %')
)
recession_chart+mortgage_chart.properties(
    width=500,
    height=500,
    title=alt.TitleParams(text = '15 yrs Firm since 1971', fontSize = 22)
)

In [ ]:
#5/1 ARM
mortgage_chart = alt.Chart(mortgage_recession_merge).mark_line().encode(
    x=alt.X('week:T', title=None),
    y=alt.Y('5_1_arm:Q', title='Mortgage Rate %')
)

recession_chart+mortgage_chart.properties(
    width=500,
    height=500,
    title=alt.TitleParams(text = '5/1 ARM since 2005', fontSize = 22)
)

In [ ]:
#from matplotlib import interactive
#30 yrs fixed, 15 yrs fixed, 5/1 ARM
melted_data = pd.melt(mortgage_recession_merge.iloc[:,np.r_[0,1,3,5]], id_vars='week', var_name='mortgage_type', value_name='rate')
line_chart = alt.Chart(melted_data).mark_line().encode(
    x='week:T',
    y='rate:Q',
    color='mortgage_type:N'
).properties(
    title='Mortgage Rates',
    width=600,
    height=400
)

recession_chart+line_chart.properties(
    width=500,
    height=500,
    title=alt.TitleParams(text = 'Fixed/Float Mortgage Rate since 2005', fontSize = 22)
)

### Home Value Index

In [ ]:
## Provide the root path to this ipynb notebook
root_path = './'

In [ ]:
#fill up missing value among the existing values
def unpivot_dataframe(file_path, Metric_value='Metric_value'):
    # read csv
    data_frame = pd.read_csv(file_path)

    # Remove unwanted columns
    data_frame = data_frame.drop(['RegionID', 'SizeRank'], axis=1)

    # Unpivot DataFrame from having date values on columns to rows
    data_frame = data_frame.melt(id_vars=['RegionName','RegionType','StateName'], var_name='Date', value_name=Metric_value)

    # Convert Date field to DateTime type
    data_frame['Date'] = pd.to_datetime(data_frame['Date'])

    # Use interpolation to fill missing values
    data_frame = data_frame.set_index('Date')
    data_frame = data_frame.groupby('RegionName', group_keys=False).apply(lambda x: x.interpolate(method='time'))
    data_frame = data_frame.reset_index()

    # drop only entries with NaN values in the metric column
    return data_frame.dropna(subset=[Metric_value])

In [ ]:
file_path = root_path + "/data/home_value_index.csv"

hvi_df = unpivot_dataframe(file_path, 'home_value_index')
hvi_df #each region has different time span

In [ ]:
hvi_df['home_value_index'].min()

In [ ]:
hvi_profile = ProfileReport(hvi_df, title="HVI Profiling Report")
hvi_profile

## Sale Price

In [ ]:
file_path = root_path + "/data/median_sale_price.csv"

median_sale_price_df = unpivot_dataframe(file_path, 'median_sale_price')
median_sale_price_df #each region has different time span

In [ ]:
sale_price_profile = ProfileReport(median_sale_price_df, title="Median Sale Price Profiling Report")
sale_price_profile

In [ ]:
#join HVI and sale price datasets
hvi_sale_price = reduce(lambda  left,right: pd.merge(left,right,on=['RegionName', 'Date', 'StateName', 'RegionType'], how='inner'), [hvi_df, median_sale_price_df])

In [ ]:
hvi_sale_price = hvi_sale_price[['Date','RegionName','home_value_index','median_sale_price']]

In [ ]:
hvi_sale_price_melt = pd.melt(hvi_sale_price, id_vars=['Date', 'RegionName'], value_vars=['home_value_index', 'median_sale_price'])
hvi_sale_price_melt = hvi_sale_price_melt.rename(columns={'variable': 'Observations'})
hvi_sale_price_melt

In [ ]:
regions = list(hvi_sale_price_melt['RegionName'].unique())
regions.sort()
input_dropdown = alt.binding_select(options=regions, name='Select Region ')

dropdown_selection = alt.selection_point(
    fields=['RegionName'],
    value='Ann Arbor, MI',
    bind=input_dropdown
)

alt.Chart(hvi_sale_price_melt).mark_line().add_params(
    dropdown_selection
).encode(
    x = alt.X('Date:T', title=''),
    y = alt.Y('value:Q', title=''),
    color = 'Observations:N'
).transform_filter(
    dropdown_selection
).properties(
    width=800,
    height=300,
    title="Home Value Index vs Median Sale Price"
)